In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from optuna import Trial, visualization
from hepml.core import plot_regression_tree


In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [3]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [4]:
pseudo_df = pd.read_csv('./best_sub.csv')
add_data_df = test_df[main_attrs]
add_data_df[target[0]] = pseudo_df[target[0]]

/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df = pd.concat([train_df[main_attrs + target], add_data_df], axis=0)

In [9]:
cols1 = ['lat', 'lon', 'climateregions__climateregion']

In [10]:
for lag in [1,2,3, 7, 14] :
    df[f'{target[0]}_lag{lag}'] = df.groupby(cols1)[target[0]].shift(lag)

In [11]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 64


In [13]:
test = df[main_attrs][len(train_df):]

In [15]:
X = df[main_attrs].values
y = df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5500, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)


5it [00:14,  2.82s/it]


(407088, 64) (407088,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 6999.5625 Total: 24268.3125


Learning rate set to 0.071366
0:	learn: 8.9999560	test: 9.0047203	best: 9.0047203 (0)	total: 33.3ms	remaining: 3m 3s
1:	learn: 8.3998773	test: 8.4033279	best: 8.4033279 (1)	total: 37.6ms	remaining: 1m 43s
2:	learn: 7.8379251	test: 7.8404173	best: 7.8404173 (2)	total: 41.7ms	remaining: 1m 16s
3:	learn: 7.3151907	test: 7.3166872	best: 7.3166872 (3)	total: 45.8ms	remaining: 1m 2s
4:	learn: 6.8303441	test: 6.8303036	best: 6.8303036 (4)	total: 49.9ms	remaining: 54.9s
5:	learn: 6.3770890	test: 6.3762328	best: 6.3762328 (5)	total: 54ms	remaining: 49.5s
6:	learn: 5.9544396	test: 5.9524070	best: 5.9524070 (6)	total: 58.2ms	remaining: 45.7s
7:	learn: 5.5586081	test: 5.5555260	best: 5.5555260 (7)	total: 62.3ms	remaining: 42.7s
8:	learn: 5.1944573	test: 5.1912457	best: 5.1912457 (8)	total: 66.4ms	remaining: 40.5s
9:	learn: 4.8529736	test: 4.8498456	best: 4.8498456 (9)	total: 70.6ms	remaining: 38.8s
10:	learn: 4.5362861	test: 4.5324658	best: 4.5324658 (10)	total: 74.8ms	remaining: 37.3s
11:	learn: 

111:	learn: 0.4245142	test: 0.4235643	best: 0.4235643 (111)	total: 652ms	remaining: 31.4s
112:	learn: 0.4233159	test: 0.4223837	best: 0.4223837 (112)	total: 657ms	remaining: 31.3s
113:	learn: 0.4223008	test: 0.4214266	best: 0.4214266 (113)	total: 666ms	remaining: 31.5s
114:	learn: 0.4213738	test: 0.4205117	best: 0.4205117 (114)	total: 673ms	remaining: 31.5s
115:	learn: 0.4205114	test: 0.4196450	best: 0.4196450 (115)	total: 678ms	remaining: 31.5s
116:	learn: 0.4195599	test: 0.4187060	best: 0.4187060 (116)	total: 682ms	remaining: 31.4s
117:	learn: 0.4186330	test: 0.4178585	best: 0.4178585 (117)	total: 687ms	remaining: 31.3s
118:	learn: 0.4178772	test: 0.4170836	best: 0.4170836 (118)	total: 691ms	remaining: 31.2s
119:	learn: 0.4167392	test: 0.4159469	best: 0.4159469 (119)	total: 696ms	remaining: 31.2s
120:	learn: 0.4159314	test: 0.4151809	best: 0.4151809 (120)	total: 700ms	remaining: 31.1s
121:	learn: 0.4148835	test: 0.4141739	best: 0.4141739 (121)	total: 704ms	remaining: 31s
122:	learn: 

...

5085:	learn: 0.1626929	test: 0.1793178	best: 0.1793178 (5085)	total: 23.3s	remaining: 1.9s
5086:	learn: 0.1626811	test: 0.1793106	best: 0.1793106 (5086)	total: 23.3s	remaining: 1.89s
5087:	learn: 0.1626751	test: 0.1793063	best: 0.1793063 (5087)	total: 23.3s	remaining: 1.89s
5088:	learn: 0.1626648	test: 0.1793015	best: 0.1793015 (5088)	total: 23.3s	remaining: 1.88s
5089:	learn: 0.1626561	test: 0.1792954	best: 0.1792954 (5089)	total: 23.3s	remaining: 1.88s
5090:	learn: 0.1626498	test: 0.1792894	best: 0.1792894 (5090)	total: 23.3s	remaining: 1.87s
5091:	learn: 0.1626432	test: 0.1792846	best: 0.1792846 (5091)	total: 23.3s	remaining: 1.87s
5092:	learn: 0.1626280	test: 0.1792700	best: 0.1792700 (5092)	total: 23.3s	remaining: 1.86s
5093:	learn: 0.1626159	test: 0.1792585	best: 0.1792585 (5093)	total: 23.3s	remaining: 1.86s
5094:	learn: 0.1626100	test: 0.1792550	best: 0.1792550 (5094)	total: 23.3s	remaining: 1.85s
5095:	learn: 0.1625984	test: 0.1792459	best: 0.1792459 (5095)	total: 23.3s	remain

5179:	learn: 0.1618642	test: 0.1786537	best: 0.1786537 (5179)	total: 23.7s	remaining: 1.46s
5180:	learn: 0.1618562	test: 0.1786479	best: 0.1786479 (5180)	total: 23.7s	remaining: 1.46s
5181:	learn: 0.1618481	test: 0.1786414	best: 0.1786414 (5181)	total: 23.7s	remaining: 1.46s
5182:	learn: 0.1618429	test: 0.1786367	best: 0.1786367 (5182)	total: 23.7s	remaining: 1.45s
5183:	learn: 0.1618293	test: 0.1786268	best: 0.1786268 (5183)	total: 23.7s	remaining: 1.45s
5184:	learn: 0.1618204	test: 0.1786201	best: 0.1786201 (5184)	total: 23.8s	remaining: 1.44s
5185:	learn: 0.1618094	test: 0.1786100	best: 0.1786100 (5185)	total: 23.8s	remaining: 1.44s
5186:	learn: 0.1618057	test: 0.1786077	best: 0.1786077 (5186)	total: 23.8s	remaining: 1.43s
5187:	learn: 0.1617976	test: 0.1786009	best: 0.1786009 (5187)	total: 23.8s	remaining: 1.43s
5188:	learn: 0.1617889	test: 0.1785963	best: 0.1785963 (5188)	total: 23.8s	remaining: 1.42s
5189:	learn: 0.1617752	test: 0.1785837	best: 0.1785837 (5189)	total: 23.8s	remai

5303:	learn: 0.1608509	test: 0.1778218	best: 0.1778218 (5303)	total: 24.3s	remaining: 899ms
5304:	learn: 0.1608452	test: 0.1778179	best: 0.1778179 (5304)	total: 24.3s	remaining: 894ms
5305:	learn: 0.1608419	test: 0.1778150	best: 0.1778150 (5305)	total: 24.3s	remaining: 890ms
5306:	learn: 0.1608282	test: 0.1778035	best: 0.1778035 (5306)	total: 24.3s	remaining: 885ms
5307:	learn: 0.1608145	test: 0.1777931	best: 0.1777931 (5307)	total: 24.3s	remaining: 881ms
5308:	learn: 0.1608094	test: 0.1777908	best: 0.1777908 (5308)	total: 24.3s	remaining: 876ms
5309:	learn: 0.1607966	test: 0.1777815	best: 0.1777815 (5309)	total: 24.4s	remaining: 871ms
5310:	learn: 0.1607871	test: 0.1777771	best: 0.1777771 (5310)	total: 24.4s	remaining: 867ms
5311:	learn: 0.1607835	test: 0.1777737	best: 0.1777737 (5311)	total: 24.4s	remaining: 862ms
5312:	learn: 0.1607778	test: 0.1777699	best: 0.1777699 (5312)	total: 24.4s	remaining: 858ms
5313:	learn: 0.1607724	test: 0.1777649	best: 0.1777649 (5313)	total: 24.4s	remai

5436:	learn: 0.1597927	test: 0.1769992	best: 0.1769992 (5436)	total: 24.9s	remaining: 289ms
5437:	learn: 0.1597863	test: 0.1769956	best: 0.1769956 (5437)	total: 25s	remaining: 284ms
5438:	learn: 0.1597767	test: 0.1769868	best: 0.1769868 (5438)	total: 25s	remaining: 280ms
5439:	learn: 0.1597701	test: 0.1769828	best: 0.1769828 (5439)	total: 25s	remaining: 275ms
5440:	learn: 0.1597575	test: 0.1769739	best: 0.1769739 (5440)	total: 25s	remaining: 271ms
5441:	learn: 0.1597524	test: 0.1769724	best: 0.1769724 (5441)	total: 25s	remaining: 266ms
5442:	learn: 0.1597459	test: 0.1769678	best: 0.1769678 (5442)	total: 25s	remaining: 262ms
5443:	learn: 0.1597393	test: 0.1769661	best: 0.1769661 (5443)	total: 25s	remaining: 257ms
5444:	learn: 0.1597296	test: 0.1769573	best: 0.1769573 (5444)	total: 25s	remaining: 252ms
5445:	learn: 0.1597217	test: 0.1769512	best: 0.1769512 (5445)	total: 25s	remaining: 248ms
5446:	learn: 0.1597112	test: 0.1769460	best: 0.1769460 (5446)	total: 25s	remaining: 243ms
5447:	le

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:48<00:00, 33.73s/it]


In [17]:
preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test.values))
s = 0
for pred in preds:
    s += pred
s = s/5
df = pd.read_csv('./sample_solution.csv')

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
t= s * 0.1 + y_best * 0.9
df['contest-tmp2m-14d__tmp2m'] = t
df.to_csv('submission_pseudo_labeling_v2.csv', index=False)

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, t)/(norm(y_best)*norm(t))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.30it/s]

cos_sim with best submission: 0.9999995033995156
